### Compare text extraction modules

In [11]:

import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import re
import pytesseract
from pdf2image import convert_from_path
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'# convert PDF to image
from PIL import Image
import fitz

In [12]:
path="reports\\2016_adidas_sustainability_progress_report.pdf"

In [13]:

reader = PdfReader(path)
number_of_pages = len(reader.pages)
text=reader.pages[0].extract_text()
for i in range(1,number_of_pages):
    page = reader.pages[i]
    text += page.extract_text()
paragraphs=re.split("\.\n|\. \n", text)
df=pd.DataFrame(np.array(paragraphs), columns=['paragraphs'])
df=df[df['paragraphs'].map(len) > 100].reset_index(drop=True)
df.to_csv("Compare\\PdfReader.csv",index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   paragraphs  286 non-null    object
dtypes: object(1)
memory usage: 2.4+ KB


In [14]:
images = convert_from_path(path) #convert pdf to images
ocr_text = ''
for i in range(len(images)):  #turn images into strings and add to ocr_text list      
    page_content = pytesseract.image_to_string(images[i])
    page_content = '***PDF Page {}***\n'.format(i+1) + page_content
    ocr_text = ocr_text + ' ' + page_content
paragraphs=re.split("\n\n", ocr_text) # split string into paragraphs
df=pd.DataFrame(np.array(paragraphs), columns=['paragraphs'])
df=df[df['paragraphs'].map(len) > 100].reset_index(drop=True) # remove short lines 
df.to_csv("Compare\\pytesseract.csv",index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315 entries, 0 to 314
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   paragraphs  315 non-null    object
dtypes: object(1)
memory usage: 2.6+ KB


In [15]:
text_list=[]
# Open the PDF file
doc = fitz.open(path)

# Loop through each page of the PDF file
for page in doc:
    for block in page.get_text("blocks"):
        # Check if block is text
        if block[6] == 0:
            # Extract text and page number
            text_list.append([block[4],page.number+1])
# Close the PDF file
doc.close()
df=pd.DataFrame(text_list,columns=["text","page_number"])
df=df[df['text'].map(len) > 100].reset_index(drop=True)
df.to_csv("Compare\\PyMyPDF.csv",index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         366 non-null    object
 1   page_number  366 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB
